In [ ]:
import torch
from torch import nn

In [ ]:
class DepSepConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.depthwise = nn.Sequential(nn.Conv2d(in_channels,in_channels,3, stride = stride, padding = 1, groups = in_channels, bias=False),
                                       nn.BatchNorm2d(in_channels),
                                       nn.ReLU(inplace=True))

        self.pointwise = nn.Sequential(nn.Conv2d(in_channels,out_channels,1, bias=False),
                                       nn.BatchNorm2d(out_channels),
                                       nn.ReLU(inplace=True))
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class MobileNet(nn.Module):
    def __init__(self, alpha, num_classes=1000):
        super().__init__()

        self.conv1 = nn.Conv2d(3, int(32*alpha), 3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(int(32*alpha))
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = DepSepConv(int(32*alpha), int(64*alpha), stride=1)
        self.conv3 = nn.Sequential(DepSepConv(int(64*alpha), int(128*alpha), stride=2), # down sample
                                   DepSepConv(int(128*alpha), int(128*alpha)))
        self.conv4 = nn.Sequential(DepSepConv(int(128*alpha), int(256*alpha), stride=2), # down sample
                                   DepSepConv(int(256*alpha), int(256*alpha)))
        self.conv5 = nn.Sequential(DepSepConv(int(256*alpha), int(512*alpha), stride=2), # down sample
                                   *[DepSepConv(int(512*alpha), int(512*alpha)) for i in range(5)])
        self.conv6 = nn.Sequential(DepSepConv(int(512*alpha), int(1024*alpha), stride=2), # down sample
                                   DepSepConv(int(1024*alpha), int(1024*alpha)))

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(int(1024*alpha), num_classes)

        # weights initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [ ]:
model = MobileNet(alpha=1)
# print(model)
!pip install torchinfo
from torchinfo import summary
summary(model, input_size=(2,3,224,224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
MobileNet                                [2, 1000]                 --
├─Conv2d: 1-1                            [2, 32, 112, 112]         864
├─BatchNorm2d: 1-2                       [2, 32, 112, 112]         64
├─ReLU: 1-3                              [2, 32, 112, 112]         --
├─DepSepConv: 1-4                        [2, 64, 112, 112]         --
│    └─Sequential: 2-1                   [2, 32, 112, 112]         --
│    │    └─Conv2d: 3-1                  [2, 32, 112, 112]         288
│    │    └─BatchNorm2d: 3-2             [2, 32, 112, 112]         64
│    │    └─ReLU: 3-3                    [2, 32, 112, 112]         --
│    └─Sequential: 2-2                   [2, 64, 112, 112]         --
│    │    └─Conv2d: 3-4                  [2, 64, 112, 112]         2,048
│    │    └─BatchNorm2d: 3-5             [2, 64, 112, 112]         128
│    │    └─ReLU: 3-6                    [2, 64, 112, 112]         --
├─Sequent

In [ ]:
x = torch.randn(2,3,224,224)
print(model(x).shape)

torch.Size([2, 1000])


## DepSep conv 와 그냥 conv 파라미터 비교

In [ ]:
print(nn.Conv2d(3,16,3).weight.numel())
print(nn.Conv2d(3,3,3,groups=3).weight.numel())
print(nn.Conv2d(3,16,1).weight.numel())

432
27
48
